In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (roc_auc_score, recall_score, precision_score, 
confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef, f1_score)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from random import randint, shuffle
import time
from datetime import datetime

def display_metrics(probs, targets, history, threshold=0.5):
    """
    Displays model performance metrics, confusion matrix, and a histogram of 
    predicted scores of a binary TensorFlow classifier model.
        
    Args:
        probs (tf.Tensor): druggability probabilities assigned by the model
        targets (pd.Series or other list-like): true labels
        history (tf.History): model training history
        threshold (float): druggability score required to designate a protein as 
            druggable

    Returns:
        None
    
    """
    
    # Displays plots of the model's loss and AUC over the training epochs for 
    # the training and validation sets
    
    print("Neural network performance")

    plt.plot(history.history['loss'], label='Train loss')
    plt.plot(history.history['val_loss'], label='validation loss')
    plt.xlabel('Epochs'), plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.plot(history.history['auc'], label='Train AUC')
    plt.plot(history.history['val_auc'], label='validation AUC')
    plt.xlabel('Epochs'), plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    
    # preds are predicted class labels obtained from the class probabilities
    
    preds = np.array([1 if x >= threshold else 0 for x in probs])
    
    
    # Computes and prints performance metrics for the model
        
    true_neg = np.sum((targets == 0) & (preds == 0))
    false_pos = np.sum((targets == 0) & (preds == 1))

    sensitivity = 100 * recall_score(y_true=targets, y_pred=preds)
    specificity = 100 * true_neg / (true_neg + false_pos)
    accuracy = 100 * sum(preds == targets) / len(targets)
    auc = 100 * roc_auc_score(y_true=targets, y_score=probs)

    print("This model:")
    print("SN:",
              round(sensitivity, 2),
              '\t',
              "SP:",
              round(specificity, 2),
              '\t',
              "ACC:",
              round(accuracy, 2),
              '\t',
              "AUC:",
              round(auc, 2),
             )
    
    
    # Generates and displays the confusion matrix

    cm = confusion_matrix(targets, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                                  display_labels=["No drug", "Drug"])
    disp.plot()
    plt.show()
    
    probs = np.array(probs)
    
    
    # Displays a histogram of class probabilities from 0 to 1, with positives 
    # and negatives separated
            
    pos_probs = np.array([probs[x] for x in range(len(probs)) 
                          if targets.iloc[x] == 1]).flatten()
    neg_probs = np.array([probs[x] for x in range(len(probs)) 
                          if targets.iloc[x] == 0]).flatten()
        
    plt.hist([pos_probs, neg_probs], 
             alpha=0.5, 
             bins=48, 
             density=True, 
             histtype = 'stepfilled', 
             label=['Drugged proteins', 'Undrugged proteins'])
    plt.xlabel('Druggability score')
    plt.legend()
    plt.show()
            
    return None
                                   
                                   
def fit_multiscore_nn(train_inputs, 
                      train_labels, 
                      validation_inputs, 
                      validation_labels):
    """
    Creates and trains the neural network classifier model and returns the model
    along with its training history.
    
    Args:
        train_inputs (list of DataFrames): List of the inputs to each of the 4 
            networks as DataFrames for the training set
        train_labels (pd.Series or other list-like): Labels for the training 
            data
        validation_inputs (list of DataFrames): List of the inputs to each of 
            the 4 networks as DataFrames for the validation set
        validation_labels (pd.Series or other list-like): Labels for the 
            validation data

    Returns:
        model (tf.keras.Model): Trained model
        history (tf.History): Model training history
        
    """
    
    # Generates the subscore for the sequence and structure subnetwork
        
    seq_and_struc_input = keras.Input(shape=(122,))
    seq_and_struc = layers.Dense(units=64, 
                                 activation='relu', 
                                 kernel_regularizer=l2)(seq_and_struc_input)
    seq_and_struc = layers.Dense(units=1, kernel_regularizer=l2)(seq_and_struc)
    
    
    # Generates the subscore for the localization subnetwork
    
    localization_input = keras.Input(shape=(558,))
    localization = layers.Dense(units=512, 
                                activation='relu',
                                kernel_regularizer=l2)(localization_input)
    localization = layers.Dense(units=1, kernel_regularizer=l2)(localization)
    
    
    # Generates the subscore for the biological functions subnetwork
    
    bio_func_input = keras.Input(shape=(3464,))
    bio_func = layers.Dense(units=2048, 
                            activation='relu', 
                            kernel_regularizer=l2)(bio_func_input)
    bio_func = layers.Dense(units=1, kernel_regularizer=l2)(bio_func)
    
    
    # Generates the subscore for the network information subnetwork
    
    network_info_input = keras.Input(shape=(8,))
    network_info = layers.Dense(units=8, 
                                activation='relu', 
                                kernel_regularizer=l2)(network_info_input)
    network_info = layers.Dense(units=1, kernel_regularizer=l2)(network_info)
    
    
    # Generates the final output from the 4 subscores
    
    subscores = layers.Concatenate(name='subscores')([seq_and_struc, 
                                                      localization, 
                                                      bio_func, 
                                                      network_info])
        
    model_output = tf.math.reduce_sum(subscores, axis=1)
    
    model = keras.Model(inputs=[seq_and_struc_input, 
                                localization_input, 
                                bio_func_input, 
                                network_info_input], 
                        outputs=model_output)
    
    
    # Compiles and trains the model

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    model.compile(optimizer=Adam(learning_rate=10**-3.5), loss=bce, 
                  metrics=tf.keras.metrics.AUC(name="auc", from_logits=True))
    
    model.summary()
    
    with tf.device('/device:GPU:0'):
        history = model.fit(x=train_inputs, 
                            y=train_labels,
                            batch_size=32, 
                            epochs=200, 
                            shuffle=True,
                            validation_data=(validation_inputs, validation_labels),
                            callbacks=[callback],
                            verbose=verbose)
    
    return model, history


def k_split(input_set, k):
    """
    Splits input data into k equally-sized sets.
    
    Args:
        input_set (DataFrame): data to split
        k (int): number of splits
        
    Returns:
        split_sets (list of DataFrames): list containing k split dataframes
        
    """
    
    input_len = len(input_set)
    
    next_n = 0
        
    k_assigns = np.array([])
        
    for i in range(input_len):
        if next_n == k:
            next_n = 0

        k_assigns = np.append(k_assigns, next_n)

        next_n += 1
        
    shuffle(k_assigns)

    input_set['k'] = k_assigns

    split_sets = []

    for i in range(k):
        set_k = input_set[input_set['k'] == i].drop('k', axis=1)

        split_sets.append(set_k)
        
    return split_sets


def process_set(protein_set):
    """
    Accepts a set of proteins, including both features and labels, and unpacks 
    into protein names, features separated into different categories, and 
    labels.

    Args:
        protein_set (DataFrame): dataset to be unpacked
        
    Returns:
        names (pd.Series): protein UniProt IDs
        seq_and_struc (DataFrame): sequence and structure features
        localization (DataFrame): localization features
        bio_func (DataFrame): biological function features
        network_info (DataFrame): network information features
        labels (pd.Series): protein labels
        
    """
    
    names = protein_set['Protein']
        
    seq_and_struc = protein_set[seq_and_struc_names]
    localization = protein_set[localization_names]
    bio_func = protein_set[bio_func_names]
    network_info = protein_set[network_info_names]
    
    labels = protein_set['drugged']

    return names, seq_and_struc, localization, bio_func, network_info, labels


def vanilla_oversample(split_sets):
    """
    Accepts a list of dataframes representing a larget dataset split into k 
    parts and randomly oversamples the positive class within each dataframe to 
    generate a new frame with balanced classes.
        
    Args:
        split_sets (list of DataFrames): datasets to be balanced
        
    Returns:
        oversamples (list of DataFrames): new oversampled dataframes
        
    """
    
    oversamples = []
    
    
    # Loops through the dataframes in split_sets and randomly oversamples the 
    # positive class
    
    for i in split_sets:   
        positives = i[i['drugged'] == 1]
        negatives = i[i.Protein.isin(positives['Protein']) == False]
        positives = positives.sample(n=len(negatives), replace=True)
        
        oversamples.append(pd.concat([positives, negatives]))
            
    return oversamples


def get_false_negatives(probs, labels, protein_names):
    """
    Identifies the false negatives in list of protein IDs with their
    corresponding model-assigned druggability scores and actual labels.
        
    Args:
        probs (tf.Tensor): probabilities assigned by a model
        labels (pd.Series): true labels
        protein_names (pd.Series): protein UniProt IDs
        
    Returns:
        false_neg (list): UniProt IDs of false negatives
        
    """
    
    preds = np.round(probs).flatten()
    
    false_neg = []
    
    for i in range(len(labels)):
        if labels.iloc[i] == 1 and preds[i] == 0:
            false_neg.append(protein_names.iloc[i])
    
    return false_neg


def shuffle_frame(df):
    """
    Randomly shuffles an inputted dataframe.
        
    Args:
        df (DataFrame): dataframe to be shuffled
        
    Returns:
        df_copy (DataFrame): shuffled frame
        
    """
    
    df_copy = df.copy()
    
    df_copy = df_copy.sample(frac=1).reset_index(drop=True)
        
    return df_copy

mse = tf.keras.losses.MeanSquaredError()
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

l2 = tf.keras.regularizers.L2(l2=0.001)
l1l2 = tf.keras.regularizers.L1L2(l1=0.001, l2=0.001)

In [ ]:
# Imports the full processed dataset and the subnetwork assignments of all of 
# its features

full_set = pd.read_csv("processed_data/full_set.csv", index_col=0)

seq_and_struc_names = pd.read_csv("processed_data/seq_and_struc_names.csv", 
                                  index_col=0, 
                                  squeeze=True).tolist()
localization_names = pd.read_csv("processed_data/localization_names.csv", 
                                 index_col=0, 
                                 squeeze=True).tolist()
bio_func_names = pd.read_csv("processed_data/bio_func_names.csv", 
                             index_col=0, 
                             squeeze=True).tolist()
network_info_names = pd.read_csv("processed_data/network_info_names.csv", 
                                 index_col=0, 
                                 squeeze=True).tolist()

In [ ]:
# Separates the test data from the rest of the dataset
# Randomly splits the training and validation sets k ways for cross-validation

test_set = full_set[full_set.test == 1].drop('test', axis=1)

train_validation_set = full_set[full_set.Protein.isin(test_set['Protein']) == 
                          False].drop('test', axis=1)

k = 5

split_sets = k_split(train_validation_set, k)

In [ ]:
start_time = time.time()

verbose = True

aucs = []
final_validation_losses = []


# n_trials is the number of cross-validation trials to perform
# Must be <= k, as each trial uses the next set in the k split as the validation 
# set

n_trials = 5

oversamples = vanilla_oversample(split_sets)

all_false_negs = []


# Iterates through the split sets, using the set at index i as the validation 
# set and all the others as the training set

for i in range(n_trials):
    
    train_sets = [oversamples[x] for x in range(k) if x != i]
                    
    train_set = pd.concat(train_sets)
    
    validation_set = split_sets[i]
        
    (train_names, train_seq_and_struc, train_localization, train_bio_func, 
    train_network_info, train_labels) = process_set(train_set)
    
    (validation_names, validation_seq_and_struc, validation_localization, 
     validation_bio_func, validation_network_info, 
     validation_labels) = process_set(validation_set)
    
    train_inputs = [train_seq_and_struc, 
                    train_localization, 
                    train_bio_func, 
                    train_network_info]
    
    validation_inputs = [validation_seq_and_struc, 
                         validation_localization, 
                         validation_bio_func, 
                         validation_network_info]
                                    
    print("Split #: ", i + 1)

    model, history = fit_multiscore_nn(train_inputs, 
                                       train_labels, 
                                       validation_inputs, 
                                       validation_labels)
    
    
    # Predicts the labels of the validation set using the model

    probs = tf.keras.activations.sigmoid(model.predict(validation_inputs))    
    
    
    # Displays the metrics and graphs from the model's training
    
    display_metrics(probs, validation_labels, history)
    
    
    # Saves the model's performance as measured by validation AUC
            
    aucs.append(100 * roc_auc_score(y_true=validation_labels, y_score=probs))
    
    
    # Saves the model's performance as measured by validation loss
    
    score = model.evaluate(validation_inputs, validation_labels, verbose=False)
    
    final_validation_losses.append(score[0])
    
    
    # Gets the false negatives and adds them to the list
    
    false_negs = get_false_negatives(probs, validation_labels, validation_names)
    all_false_negs += false_negs
    

# Displays the AUC for each trial and the time elapsed in the experiment

print(aucs)
    
elapsed_time = time.time() - start_time

print("Time elapsed (s): ", elapsed_time)

In [ ]:
def get_subscores(features, model):
    """
    Accepts features for a set of proteins, divided by subnetwork, and generates
    scores for each of the subnetworks.
        
    Args:
        features (list of DataFrames): input features for each network
        model (tf.Model): model to be used to obtain subscores
        
    Returns:
        subscores_output (tf.Tensor): subscores from each of the networks
        
    """
    
    # Generates the subscores for each of the networks
    
    subscores_model = keras.Model(inputs=model.input, 
                                  outputs=model.get_layer('subscores').output)
    
    subscores_output = subscores_model.predict(features, verbose=False)
    
    total_scores = tf.math.reduce_sum(subscores_output, axis=1)
    
    
    # Generates the final druggability probability and combines it with the
    # subscores
    
    drug_probs = tf.keras.activations.sigmoid(
        tf.reshape(total_scores, shape=(len(total_scores),1))) * 100

    subscores_output = tf.concat([drug_probs, subscores_output], axis=1)
            
    return subscores_output